In [1]:
import sys
sys.path.insert(0, '../')

import pandas as pd
from sklearn import metrics
#from keras.wrappers.scikit_learn import KerasClassifier
from classification_plots import plot_confusion_matrix
import matplotlib.pyplot as plt
import os
from collections import Counter
import numpy as np
import scipy 
%matplotlib notebook

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import warnings
warnings.filterwarnings('ignore')

In [3]:
audio_dir = '../../ESC-50/audio/'
path_to_db='../../ESC-50/'

In [4]:
from torch.utils.data import Dataset, DataLoader
import librosa
fs = 44100
class audiotrainset(Dataset):
    def __init__(self):
        
        root = "./data1/"
        self.data_list = []
        self.label_list = []
        for root, dir, files in os.walk("./data1/"):
            for file in files:
                if file.find('.wav')!= -1:
#                     print(os.path.join(root, file))
#                     print(int(root[13:]))
                    self.data_list.append(os.path.join(root, file))
                    self.label_list.append(int(root[13:]))
        print(len(self.data_list), len(self.label_list))
        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index):

        path = self.data_list[index]
        data,sr = librosa.core.load(path)
#         data = np.expand_dims(data, axis=0)
        label= self.label_list[index] 

        label = np.asarray(label)
        
        return data, label,path
audiodataloader = audiotrainset()
data,label,path = audiodataloader.__getitem__(895)
print(path)

2000 2000
./data1/class29\2-94807-A-29.wav


In [5]:
## Extracts LPC coefficients ###
import numpy as np
import scipy as sp
import scipy.signal as sig

def lpc_ref(signal, order):
    """Compute the Linear Prediction Coefficients.
    Return the order + 1 LPC coefficients for the signal. c = lpc(x, k) will
    find the k+1 coefficients of a k order linear filter:
      xp[n] = -c[1] * x[n-2] - ... - c[k-1] * x[n-k-1]
    Such as the sum of the squared-error e[i] = xp[i] - x[i] is minimized.
    Parameters
    ----------
    signal: array_like
        input signal
    order : int
        LPC order (the output will have order + 1 items)
    Note
    ----
    This is just for reference, as it is using the direct inversion of the
    toeplitz matrix, which is really slow"""
    if signal.ndim > 1:
        raise ValueError("Array of rank > 1 not supported yet")
#     if order > signal.size:
#         raise ValueError("Input signal must have a lenght >= lpc order")

    if order > 0:
        p = order + 1
        r = np.zeros(p, signal.dtype)
        # Number of non zero values in autocorrelation one needs for p LPC
        # coefficients
        nx = np.min([p, signal.size])
        x = np.correlate(signal, signal, 'full')
        r[:nx] = x[signal.size-1:signal.size+order]
        phi = np.dot(sp.linalg.inv(sp.linalg.toeplitz(r[:-1])), -r[1:])
        return np.concatenate(([1.], phi))
    else:
        return np.ones(1, dtype = signal.dtype)

def levinson_1d(r, order):
    """Levinson-Durbin recursion, to efficiently solve symmetric linear systems
    with toeplitz structure.
    Arguments
    ---------
        r : array-like
            input array to invert (since the matrix is symmetric Toeplitz, the
            corresponding pxp matrix is defined by p items only). Generally the
            autocorrelation of the signal for linear prediction coefficients
            estimation. The first item must be a non zero real.
    Note
    ----
    This implementation is in python, hence unsuitable for any serious
    computation. Use it as educational and reference purpose only.
    Levinson is a well-known algorithm to solve the Hermitian toeplitz
    equation:
                       _          _
        -R[1] = R[0]   R[1]   ... R[p-1]    a[1]
         :      :      :          :      *  :
         :      :      :          _      *  :
        -R[p] = R[p-1] R[p-2] ... R[0]      a[p]
                       _
    with respect to a (  is the complex conjugate). Using the special symmetry
    in the matrix, the inversion can be done in O(p^2) instead of O(p^3).  
    """
    r = np.atleast_1d(r)
    if r.ndim > 1:
        raise ValueError("Only rank 1 are supported for now.")

    n = r.size
    if order > n - 1:
        raise ValueError("Order should be <= size-1")
    elif n < 1:
        raise ValueError("Cannot operate on empty array !")

    if not np.isreal(r[0]):
        raise ValueError("First item of input must be real.")
    elif not np.isfinite(1/r[0]):
        raise ValueError("First item should be != 0")

    # Estimated coefficients
    a = np.empty(order+1, r.dtype)
    # temporary array
    t = np.empty(order+1, r.dtype)
    # Reflection coefficients
    k = np.empty(order, r.dtype)

    a[0] = 1.
    e = r[0]

    for i in xrange(1, order+1):
        acc = r[i]
        for j in range(1, i):
            acc += a[j] * r[i-j]
        k[i-1] = -acc / e
        a[i] = k[i-1]

        for j in range(order):
            t[j] = a[j]

        for j in range(1, i):
            a[j] += k[i-1] * np.conj(t[i-j])

        e *= 1 - k[i-1] * np.conj(k[i-1])

    return a, e, k

In [6]:
######## Wavelet Decomposition #######
import pywt
def wavelet_decompose(y,wtype):
    wt = pywt.Wavelet(wtype)
    cA = pywt.wavedec(y,wt,level=2)[0]
    cA = np.asarray(cA)
    return cA

In [7]:
##### Stats from a signal #####
def get_stats(y):
    split_array = np.array_split(y,4)
    stats_vector = np.zeros(len(split_array)*2)
    offset = 0
    for idx,chunk in enumerate(split_array,0):
        stats_vector[offset] = np.mean(chunk)
        stats_vector[offset + 1] = np.var(chunk)
        offset = offset + 2
    stats_vector = np.asarray(stats_vector)
    return (stats_vector)
    

In [8]:
######### Extract MFCC Coefficients #########
def extract_mfcc(y,sr):
    mfcc_array = librosa.feature.mfcc(y,sr,n_mfcc=13)
    mfcc_features = []
    for i in range(13):
        mfcc_wav = wavelet_decompose(mfcc_array[i,:],'haar')
        lpc_coeffs = lpc_ref(mfcc_wav,5)[1:]
        mfcc_features.append(np.asarray(lpc_coeffs))
    
    
    final_coeffs = 0
    for coeff in mfcc_features:
        final_coeffs = np.concatenate((final_coeffs,coeff),axis = None)
    
    final_coeffs = final_coeffs[1:]
    
    final_coeffs = np.asarray(final_coeffs) 
    return (final_coeffs)

In [9]:
def feature_extraction(y):
    sr = 44100
    f1 = (lpc_ref(y,9)[1:])
    f2 = (get_stats(wavelet_decompose(y,'haar')))
    f3 = (extract_mfcc(y,sr))
    f4 = (get_stats(wavelet_decompose(y,'db4')))
    f5 = (get_stats(wavelet_decompose(y,'sym4')))
    local_features =  np.concatenate((f1,f2,f3,f4,f5),axis = None)
    return local_features

In [10]:
import torch.nn as nn
import torch.nn.functional as F
class SpectralNet(nn.Module):
    def __init__(self,):
        super(SpectralNet, self).__init__()
        self.fc1 = nn.Linear(98,200)
        self.fc2 = nn.Linear(200,200)
        self.fc3 = nn.Linear(200,50)
        self.softmax = nn.Softmax()
        self.batchnorm1 = nn.BatchNorm1d(200)
        self.batchnorm2 = nn.BatchNorm1d(200)

    def forward(self, x):
        x = self.fc1(x)
        x = self.batchnorm1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.batchnorm2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

model = SpectralNet()
print (model)
model.cuda()    

SpectralNet(
  (fc1): Linear(in_features=98, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=50, bias=True)
  (softmax): Softmax(dim=None)
  (batchnorm1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


SpectralNet(
  (fc1): Linear(in_features=98, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=50, bias=True)
  (softmax): Softmax(dim=None)
  (batchnorm1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [12]:
import torch.optim as optim
import torch
from torch.utils.data.sampler import SubsetRandomSampler

device = 0
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

validation_split = .25
random_seed= 42
shuffle_dataset = True
dataset_size = 2000
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

dataset  = audiodataloader
trainloader = torch.utils.data.DataLoader(dataset, batch_size=5, 
                                           sampler=train_sampler,pin_memory=True)
valloader = torch.utils.data.DataLoader(dataset, batch_size=5,
                                                sampler=valid_sampler,pin_memory=True)


for epoch in range(400):  # loop over the dataset multiple times
    
    count = 0
    validation_accuracy = 0
    running_loss = 0.0
    training_accuracy = 0
    ######## training Stage ###########
    for i, data in enumerate(trainloader, 0):
        count += 1
        inputs, labels,paths = data
        batch_feature = np.empty((0,98))
        for path in paths:
            audio,sr = librosa.core.load(path)
            features = feature_extraction(audio)
            batch_feature = np.vstack((batch_feature,features))
            
        
        print(batch_feature.shape)
        labels = labels.long().to(device)
        batch_feature = torch.from_numpy(batch_feature).float().to(device)
        optimizer.zero_grad()

        outputs = model(batch_feature)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        acc_train = torch.eq(preds, labels).float().mean()
        training_accuracy += acc_train.item()
        if i % 100 == 99:
            print(epoch, i, running_loss/100)
            running_loss = 0
        
    epoch_loss = running_loss / count
    print("epoch loss:", epoch, epoch_loss)
    print("train_accuracy:", training_accuracy/count)
    
# #             torch.save(net, 'toy_model.pt')
        
    
    print('Finished Training')    
    running_loss = 0.0
    count = 0
    for i, data in enumerate(valloader, 0):
        count += 1
        inputs, labels,paths = data
        batch_feature = np.empty((0,98))
        for path in paths:
            audio,sr = librosa.core.load(path)
            features = feature_extraction(audio)
            batch_feature = np.vstack((batch_feature,features))
            
        
        print(batch_feature.shape)
        labels = labels.long().to(device)
        batch_feature = torch.from_numpy(batch_feature).float().to(device)
        outputs = model(batch_feature)
        _, preds = torch.max(outputs, 1)
        acc_val = torch.eq(preds, labels).float().mean()
        validation_accuracy += acc_val.item()
        
    print("val_accuracy:", validation_accuracy/count)

(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)
(5, 98)


KeyboardInterrupt: 